In [1]:
import pandas as pd

data=pd.read_excel(r'''C:\GitHub Upload\TelecomCustChurn - 0 - Raw Data.xlsx''',  
                   sheetname='clean')

data.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,tenure_tiers,tenure_0-3mos,tenure_3-6mos,...,MonthlyCharges,MonthlyCharges_tiers,MonthlyCharges_0-19.99,MonthlyCharges_20-39.99,MonthlyCharges_40-59.99,MonthlyCharges_60-79.99,MonthlyCharges_80-99.99,MonthlyCharges_100-119.99,TotalCharges,Blend
0,2923-ARZLG,0,1,0,1,1,0,0-3mos,1,0,...,19.70,0-19.99,1,0,0,0,0,0,,1
1,7644-OMVMY,0,1,0,1,1,0,0-3mos,1,0,...,19.85,0-19.99,1,0,0,0,0,0,,1
2,2520-SGTTA,0,0,0,1,1,0,0-3mos,1,0,...,20.00,20-39.99,0,1,0,0,0,0,,1
3,3115-CZMZD,0,1,0,0,1,0,0-3mos,1,0,...,20.25,20-39.99,0,1,0,0,0,0,,1
4,3213-VVOLG,0,1,0,1,1,0,0-3mos,1,0,...,25.35,20-39.99,0,1,0,0,0,0,,1


In [2]:
import math
import numpy as np
import pandas as pd
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import cross_val_score, cross_val_predict, StratifiedKFold 
from sklearn import preprocessing, metrics, svm, ensemble
from sklearn.metrics import accuracy_score, classification_report
import tabpy_client 

C:\Users\groldan\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\groldan\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
y=data['Churn']

X=data[['Contract_mon_to_mon', 'InternetService_FiberOp', 'PaymentMethod_echeck', 'PaperlessBilling', 'OnlineSecurity_adj', 
       'TechSupport_adj', 'Dependents', 'SeniorCitizen', 'Partner']]

# Scale the data. We will use the same scaler later for scoring function
# scaler = preprocessing.StandardScaler().fit(X)
# X = scaler.transform(X)

# 10 fold stratified cross validation
kf = StratifiedKFold(y, n_folds=10, random_state=None, shuffle=True)

# Logistic regression with 10 fold stratified cross-validation using model specific cross-validation in scikit-learn
lgclf = LogisticRegressionCV(Cs=list(np.power(10.0, np.arange(-10, 10))),penalty='l2',scoring='roc_auc',cv=kf) #penalty L2 not 12
lgclf.fit(X, y)
Churn_pred = lgclf.predict(X)

# Show classification report for the best model (set of parameters) run over the full dataset
print("Classification report:")
print(classification_report(y, Churn_pred))

# Show accuracy and area under ROC curve
print("Accuracy: %0.3f" % accuracy_score(y, Churn_pred, normalize=True))
print("Aucroc: %0.3f" % metrics.roc_auc_score(y, Churn_pred))

Classification report:
             precision    recall  f1-score   support

          0       0.82      0.89      0.86      5174
          1       0.61      0.48      0.53      1869

avg / total       0.77      0.78      0.77      7043

Accuracy: 0.780
Aucroc: 0.683


In [ ]:
def ChurnProb(ContractM2M, FiberOp, eCheck, PaperlessBill, OnlineSec, TechSupp, Dependent, Senior, Partner):
    X2=np.column_stack([ContractM2M, FiberOp, eCheck, PaperlessBill, OnlineSec, TechSupp, Dependent, Senior, Partner])
    
    list=lgclf.predict_proba(X2)
    list=list[:, 1]

    return list.tolist()

# Connect to TabPy server using the client library
connection = tabpy_client.Client('http://localhost:9004/')
#connection = tabpy_client.Client('http://10.107.72.24:9004/api')

connection.deploy('ChurnProb',
                  ChurnProb,
                  "What is the customer's probability of churning?", override='True')


In [ ]:

connection.remove('ChurnProb')

In [ ]:
from collections import Counter

Counter(Churn_pred)

In [ ]:
data['Churn_pred']=Churn_pred

all=data[['Contract_mon_to_mon', 'InternetService_FiberOp', 'PaymentMethod_echeck', 'PaperlessBilling', 'OnlineSecurity_adj', 
       'TechSupport_adj', 'Dependents', 'SeniorCitizen', 'Partner', 'Churn', 'Churn_pred']]

In [ ]:
#all.to_csv(r'''C:\Users\groldan\Desktop\My Learning\Springboard\_TelecomCustChurn_Analysis\del.csv''')b


In [ ]:
ChurnProb(1,1,1,1,0,0,0,0,0)

1. Exclude duration variables (e.g. tenure) from model since there is multicollinearity between those variables and churn.
2. Create unique models for Fiber Optic and DSL customers?
3. Tune the model to be more accurate for predicting churners instead of non-churners.